In [15]:
import numpy as np
import pandas as pd
import re

In [16]:
dataset = pd.read_csv('job_dataset.csv')
dataset.head()

,JobID,Title,ExperienceLevel,YearsOfExperience,Skills,Responsibilities,Keywords
0,NET-F-001,.NET Developer,Fresher,0-1,C#; VB.NET basics; .NET Framework; .NET Core f...,Assist in coding and debugging applications; L...,.NET; C#; ASP.NET MVC; Entity Framework; SQL S...
1,NET-F-002,.NET Developer,Fresher,0-1,C#; .NET Framework basics; ASP.NET; Razor; HTM...,Write simple C# programs under guidance; Suppo...,.NET; C#; ASP.NET MVC; Entity Framework; SQL S...
2,NET-F-003,.NET Developer,Fresher,0-1,C#; VB.NET basics; .NET Core; ASP.NET MVC; HTM...,Contribute to development of small modules; As...,.NET; C#; ASP.NET MVC; SQL Server; Entity Fram...
3,NET-F-004,.NET Developer,Fresher,0-1,C#; .NET Framework; ASP.NET basics; SQL Server...,Support in software design documentation; Assi...,.NET; C#; SQL Server; Entity Framework; ASP.NET
4,NET-F-005,.NET Developer,Fresher,0-1,C#; ASP.NET; MVC; Entity Framework basics; SQL...,Learn to design and build ASP.NET applications...,.NET; C#; ASP.NET MVC; Entity Framework; SQL S...


In [17]:
dataset = dataset.drop(['JobID', 'ExperienceLevel', 'YearsOfExperience', 'Responsibilities', 'Keywords'], axis = 1)
dataset.head(2)

,Title,Skills
0,.NET Developer,C#; VB.NET basics; .NET Framework; .NET Core f...
1,.NET Developer,C#; .NET Framework basics; ASP.NET; Razor; HTM...


In [26]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
dataset['Title'] = encoder.fit_transform(dataset['Title'])
dataset['Title'].unique()

array([  0,   2,   1, 218,   3,   8,  95,   5,  72,  14, 190,  96,  10,
         9,   7, 153, 109,  12, 135,  13, 154,   6,  11, 136,   4,  23,
        24,  25,  26,  27,  22,  21,  28,  42,  97,  43,  15,  32, 155,
       110,  41,  62,  35, 127,  37,  29,  39,  31,  66,  87,  30,  36,
        34,  33,  44,  45,  47, 103,  48,  16,  71, 192,  50,  84,  51,
        89, 162, 151, 111,  49,  40,  90,  46, 157, 140,  53,  52,  55,
        59,  98,  54,  61, 100,  68,  57,  76,  56,  60, 156, 112, 137,
       124, 160, 115,  58,  64,  63,  65,  74,  75,  78,  79,  80,  81,
        82,  86, 215, 108, 208,  73,  85, 194, 104, 212,  20, 213, 211,
       209, 165, 119, 214, 216, 210, 217, 142, 166,  92,  94,  93, 126,
       125, 123, 120,  77,  69, 121, 122, 114, 138, 159, 113, 101, 133,
        17, 191, 131, 102,  83,  70, 130,  88, 129, 161, 132,  38, 116,
        91, 128,  67, 139, 187, 134, 143, 144, 145, 146, 148, 147, 149,
       152, 150, 167, 168, 169, 170, 171, 172, 173, 175, 105, 18

In [19]:
dataset.isna().sum()

Title     0
Skills    0
dtype: int64

In [20]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Skills'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to C:\Users\Vijayabhaskar
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
print(corpus)

['c vb net basic net framework net core fundament asp net mvc html css javascript basic sql server entiti framework basic linq visual studio git unit test basic', 'c net framework basic asp net razor html css javascript basic sql server entiti framework basic nunit basic', 'c vb net basic net core asp net mvc html css javascript basic sql server git', 'c net framework asp net basic sql server html css linq visual studio', 'c asp net mvc entiti framework basic sql server html css javascript basic', 'c net framework asp net razor sql server git nunit basic', 'c vb net net core basic asp net mvc html css sql server visual studio', 'c net framework basic asp net mvc entiti framework basic sql server linq', 'c net framework asp net html css sql server', 'c asp net mvc net core basic entiti framework basic sql server git', 'c net core asp net mvc sql server entiti framework rest api docker azur', 'c asp net core sql server entiti framework design pattern solid principl azur devop', 'c f net 

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 0].values

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [24]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [25]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[144 144]
 [ 32  31]
 [207 171]
 [  4   4]
 [149 149]
 [ 25  25]
 [  4   4]
 [ 86  86]
 [ 93  93]
 [125 125]
 [ 26  26]
 [143 143]
 [188 188]
 [ 44  45]
 [186 186]
 [  3   3]
 [189 189]
 [ 23  23]
 [175 180]
 [ 58 160]
 [ 21  21]
 [ 53  53]
 [ 82  82]
 [ 75  75]
 [195 195]
 [ 64  64]
 [206 206]
 [ 46 157]
 [172 172]
 [124 112]
 [ 74  74]
 [ 45  45]
 [207 207]
 [ 94  94]
 [ 79  79]
 [150 150]
 [  1   1]
 [172 172]
 [ 79  79]
 [ 12   7]
 [134 134]
 [189 189]
 [ 33  33]
 [186 186]
 [150 150]
 [167 167]
 [149 149]
 [ 21  21]
 [172 172]
 [195 195]
 [172 172]
 [170 170]
 [ 75  75]
 [152 152]
 [ 55  55]
 [186 186]
 [145 145]
 [ 64  64]
 [148 148]
 [150 150]
 [143 164]
 [ 24  24]
 [ 88 191]
 [ 40 151]
 [108  20]
 [ 82  82]
 [ 55  55]
 [143 198]
 [186 186]
 [  3   3]
 [195 107]
 [ 45  45]
 [144 144]
 [206 206]
 [132  50]
 [  2   2]
 [ 22  22]
 [152 152]
 [ 82  82]
 [ 86  86]
 [150 150]
 [ 55  55]
 [ 76  54]
 [ 23  23]
 [207 207]
 [144 144]
 [ 27  27]
 [152 152]
 [143 143]
 [124 115]
 [134 134]

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[4 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


0.8037383177570093

In [31]:
new_input = ["AWS basics; Azure basics; GCP basics; Terraform/CloudFormation knowledge; Docker/Kubernetes; Python/Bash scripting; Jenkins; GitHub Actions; Monitoring fundamentals; Git"]
new_input_vectorized = cv.transform(new_input).toarray()

new_prediction = classifier.predict(new_input_vectorized)

print(f"Predicted class: {new_prediction[0]}")

Predicted class: 34
